In [1]:
import pyvisa
import ipywidgets as widgets
from IPython.display import display, clear_output
import threading
import time

# Globals
stop_thread = False
inst1 = None
inst2 = None

# GUI Elements
interval_input = widgets.FloatText(description='Interval (s):', value=1.0)
count_input = widgets.IntText(description='Readings:', value=10)
query_input = widgets.Text(description='SCPI Cmd:', value='READ?')
start_button = widgets.Button(description="Start", button_style='success')
stop_button = widgets.Button(description="Stop", button_style='danger')
output_area = widgets.Output()

# VISA Setup
rm = pyvisa.ResourceManager()
resources = rm.list_resources()
inst1_dropdown = widgets.Dropdown(options=resources, description="Device 1:")
inst2_dropdown = widgets.Dropdown(options=resources, description="Device 2:")
connect_button = widgets.Button(description="Connect", button_style='primary')

def connect_devices(_):
    global inst1, inst2
    try:
        inst1 = rm.open_resource(inst1_dropdown.value)
        inst1.write_termination = '\n'
        inst1.read_termination = '\n'
        inst1.timeout = 5000
        
        inst2 = rm.open_resource(inst2_dropdown.value)
        inst2.write_termination = '\n'
        inst2.read_termination = '\n'
        inst2.timeout = 5000

        with output_area:
            clear_output()
            print("Connected to:")
            print("  Device 1:", inst1.query("*IDN?").strip())
            print("  Device 2:", inst2.query("*IDN?").strip())
    except Exception as e:
        with output_area:
            clear_output()
            print("Connection error:", e)

connect_button.on_click(connect_devices)

# Display GUI
device_selection = widgets.VBox([
    widgets.HBox([inst1_dropdown, inst2_dropdown]),
    connect_button,
    interval_input,
    count_input,
    query_input,
    widgets.HBox([start_button, stop_button]),
    output_area
])
display(device_selection)

# Continuous Read Function
def read_dual():
    global stop_thread
    stop_thread = False
    interval = interval_input.value
    count = count_input.value
    query = query_input.value

    with output_area:
        clear_output()
        print(f"Reading '{query}' from both devices...\n")
    try:
        for i in range(count):
            if stop_thread:
                break

            val1_raw = inst1.query(query).strip()
            val2_raw = inst2.query(query).strip()
            
            # Extract the first numeric value before comma
            try:
                val1 = float(val1_raw.split(',')[0])
                val2 = float(val2_raw.split(',')[0])
                diff = val1 - val2
            except (ValueError, IndexError):
                val1 = val1_raw
                val2 = val2_raw
                diff = "NaN"


            with output_area:
                print(f"[{i+1}] Device 1: {val1} | Device 2: {val2} | Δ: {diff}")

            time.sleep(interval)

        with output_area:
            print("\nReading complete.")

    except Exception as e:
        with output_area:
            print("Error:", e)

def start_read(_):
    thread = threading.Thread(target=read_dual)
    thread.start()

def stop_read(_):
    global stop_thread
    stop_thread = True
    with output_area:
        print("\nReading stopped.")

start_button.on_click(start_read)
stop_button.on_click(stop_read)


Reading 'READ?' from both devices...

[1] Device 1: 0.14879 | Device 2: 0.19723 | Δ: -0.04843999999999998
[2] Device 1: 0.14883 | Device 2: 0.19728 | Δ: -0.04845000000000002
[3] Device 1: 0.14881 | Device 2: 0.19729 | Δ: -0.048479999999999995
[4] Device 1: 0.14879 | Device 2: 0.19725 | Δ: -0.04846
[5] Device 1: 0.1488 | Device 2: 0.19725 | Δ: -0.04845000000000002
[6] Device 1: 0.14879 | Device 2: 0.19725 | Δ: -0.04846
[7] Device 1: 0.1488 | Device 2: 0.19724 | Δ: -0.04844000000000001
[8] Device 1: 0.14878 | Device 2: 0.19727 | Δ: -0.048490000000000005
[9] Device 1: 0.14877 | Device 2: 0.19728 | Δ: -0.04851
[10] Device 1: 0.14879 | Device 2: 0.19729 | Δ: -0.04849999999999999

Reading complete.
Reading 'READ?' from both devices...

[1] Device 1: 0.20164 | Device 2: 0.15234 | Δ: 0.04930000000000001
[2] Device 1: 0.20163 | Device 2: 0.1523 | Δ: 0.04933000000000001
[3] Device 1: 0.20163 | Device 2: 0.15231 | Δ: 0.04932
[4] Device 1: 0.2017 | Device 2: 0.15236 | Δ: 0.049339999999999995
[5] D